### Image Segmentation

In [ ]:
%matplotlib inline
import keras
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.datasets import mnist
from keras.utils import np_utils
from keras.preprocessing.image import img_to_array
from skimage.transform import resize

In [50]:
%run Dataset.py

In [51]:
SEG_PATH = './CityScapes Dataset/segmentations'
IMG_PATH = './CityScapes Dataset/rawimgs'
SPLIT = 'train'

In [53]:
train_gen = DataGenerator(img_path=IMG_PATH, seg_path=SEG_PATH, split='TRAIN',
                  n_classes=30,
                  input_width=224, input_height=224,
                  output_width=112, output_height=112,
                  imgNorm='sub_mean',
                  ordering='channels_first',
                  batch_size=16
                  )

In [55]:
val_gen = DataGenerator(img_path=IMG_PATH, seg_path=SEG_PATH, split='TEST',
                  n_classes=30,
                  input_width=224, input_height=224,
                  output_width=112, output_height=112,
                  imgNorm='sub_mean',
                  ordering='channels_first',
                  batch_size=16
                  )

## Image Segmentation - Preprocess Datasets

In [43]:
def VGGUNet(n_classes, weights_path, input_height, input_width, 
            image_ordering='channels_first'):
  
    assert input_height % 32 == 0
    assert input_width % 32 == 0

    img_input = Input(shape=(3, input_height, input_width))

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=image_ordering)(
        img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=image_ordering)(x)
    f1 = x
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=image_ordering)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=image_ordering)(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=image_ordering)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=image_ordering)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=image_ordering)(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=image_ordering)(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=image_ordering)(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1000, activation='softmax', name='predictions')(x)

    vgg = Model(img_input, x)
    vgg.load_weights(weights_path)

    o = f4

    o = (ZeroPadding2D((1, 1), data_format=image_ordering))(o)
    o = (Conv2D(512, (3, 3), padding='valid', data_format=image_ordering))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=image_ordering))(o)
    o = (concatenate([o, f3], axis=1))
    o = (ZeroPadding2D((1, 1), data_format=image_ordering))(o)
    o = (Conv2D(256, (3, 3), padding='valid', data_format=image_ordering))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=image_ordering))(o)
    o = (concatenate([o, f2], axis=1))
    o = (ZeroPadding2D((1, 1), data_format=image_ordering))(o)
    o = (Conv2D(128, (3, 3), padding='valid', data_format=image_ordering))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=image_ordering))(o)
    o = (concatenate([o, f1], axis=1))
    o = (ZeroPadding2D((1, 1), data_format=image_ordering))(o)
    o = (Conv2D(64, (3, 3), padding='valid', data_format=image_ordering))(o)
    o = (BatchNormalization())(o)

    o = Conv2D(n_classes, (3, 3), padding='same', data_format=image_ordering)(o)
    o_shape = Model(img_input, o).output_shape
    outputHeight = o_shape[2]
    outputWidth = o_shape[3]

    o = (Reshape((n_classes, outputHeight * outputWidth)))(o)
    o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model(img_input, o)
    model.outputWidth = outputWidth
    model.outputHeight = outputHeight

    return model

### Model Configurations

In [56]:
IMAGE_ORDERING = 'channels_first'
VGG_WEIGHTS_PATH = "vgg16_weights_th_dim_ordering_th_kernels.h5"
N_CLASSES = 30
INPUT_HEIGHT = 224
INPUT_WIDTH = 224

### Run VGG UNet

In [57]:
model = VGGUNet(N_CLASSES, VGG_WEIGHTS_PATH, INPUT_HEIGHT, INPUT_WIDTH, IMAGE_ORDERING)

In [58]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 3, 224, 224)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 64, 224, 224) 1792        input_10[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 64, 224, 224) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 64, 112, 112) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [59]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [60]:
model.fit_generator(generator=train_gen, steps_per_epoch=100, 
                validation_data=val_gen, validation_steps=200, 
                epochs=5, use_multiprocessing=True)

m.save_weights('vgg_unet_weights')

m.save('vgg_unet' + ".model")

Epoch 1/5
 54/100 [===============>..............] - ETA: 40:19 - loss: 0.4907 - acc: 0.4350

Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  

KeyboardInterrupt: 

In [0]:
base_dir = "."
X_train_path = os.path.join(base_dir, "images_prepped_train/")
y_train_path = os.path.join(base_dir, "annotations_prepped_train/")
X_val_path = os.path.join(base_dir, "images_prepped_test/")
y_val_path = os.path.join(base_dir, "annotations_prepped_test/")


batch_size = 2
epochs = 5
n_classes = 10
input_height = 224
input_width = 224
output_height = 112
output_width = 112

In [ ]:
def image_segmentation_dataset(X_path, y_path, n_classes, 
                               input_height, input_width, 
                               output_height, output_width):
  
    assert X_path[-1] == '/'
    assert y_path[-1] == '/'

    images = glob.glob(X_path + "*.jpg") + glob.glob(X_path + "*.png") + glob.glob(X_path + "*.jpeg")
    images.sort()
    segmentations = glob.glob(y_path + "*.jpg") + glob.glob(y_path + "*.png") + glob.glob(y_path + "*.jpeg")
    segmentations.sort()

    assert len(images) == len(segmentations)
    
    for im, seg in zip(images, segmentations):
        assert (im.split('/')[-1].split(".")[0] == seg.split('/')[-1].split(".")[0])
        
    print("Making datasets!")
    X = [getImageArr(im, input_width, input_height) for im in images]
    y = [getSegmentationArr(seg, n_classes, output_width, output_height) for seg in segmentations]

    return np.array(X), np.array(y)

In [0]:
X_train, y_train = image_segmentation_dataset(
    X_train_path, y_train_path, n_classes, 
    input_height, input_width, 
    output_height, output_width)

Making datasets!


In [0]:
X_test, y_test = image_segmentation_dataset(
    X_val_path, y_val_path, n_classes, 
    input_height, input_width, 
    output_height, output_width)

Making datasets!


In [0]:
X_train.shape

(367, 3, 224, 224)

In [0]:
y_train.shape

(367, 12544, 10)

In [0]:
X_test.shape

(101, 3, 224, 224)

In [0]:
y_test.shape

(101, 12544, 10)

## Create VGG U-Net Model

In [0]:
model = VGGUNet(n_classes, 
                VGG_weights_path, 
                input_height, 
                input_width)

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

## Active Learning

In [0]:
def train_test_split(X, y, percent):
    X_train = []
    y_train = []
    X_test = list(X)
    y_test = list(y)
    
    train_size = int(len(X_test) * percent)
    
    while len(X_train) < train_size:
      index = randrange(len(X_test))
      X_train.append(X_test.pop(index))
      y_train.append(y_test.pop(index))
      
    return np.array(X_test), np.array(y_test), np.array(X_train), np.array(y_train)

In [0]:
def split_dataset(X_train, X_test, y_train, y_test, initial_annotated_perc=0.1):  
    X_pool, y_pool, X_initial, y_initial = train_test_split(X_train, 
                                                            y_train, 
                                                            initial_annotated_perc)
    return X_pool, y_pool, X_initial, y_initial, X_test, y_test

In [0]:
def initialize_model(model, X_initial, y_initial, X_test, y_test, 
                     n_classes, epochs, batch_size, verbose):
    
    model.fit(X_initial, y_initial, validation_data=(X_test, y_test), 
              shuffle=True, batch_size=2, epochs=epochs, verbose=verbose)

    scores = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
    print('Initial Test Loss: ', scores[0], ' Initial Test Accuracy: ', scores[1])
    return model

In [0]:
# Random sampling
def random_sampling(y_pred_prob, n_samples):
    return np.random.choice(range(len(y_pred_prob)), n_samples)

In [0]:
# Rank all the unlabeled samples in an ascending order according to the least confidence
def least_confidence(y_pred_prob, n_samples):
    origin_index = np.arange(0, len(y_pred_prob))
    max_prob = np.max(y_pred_prob, axis=1)
    pred_label = np.argmax(y_pred_prob, axis=1)

    lci = np.column_stack((origin_index,
                           max_prob,
                           pred_label))
    lci = lci[lci[:, 1].argsort()]
    return lci[:n_samples], lci[:, 0].astype(int)[:n_samples]

In [0]:
# Rank all the unlabeled samples in an ascending order according to the margin sampling
def margin_sampling(y_pred_prob, n_samples):
    origin_index = np.arange(0, len(y_pred_prob))
    margim_sampling = np.diff(-np.sort(y_pred_prob)[:, ::-1][:, :2])
    pred_label = np.argmax(y_pred_prob, axis=1)
    msi = np.column_stack((origin_index,
                           margim_sampling,
                           pred_label))
    msi = msi[msi[:, 1].argsort()]
    return msi[:n_samples], msi[:, 0].astype(int)[:n_samples]

In [0]:
# Rank all the unlabeled samples in an descending order according to their entropy
def entropy(y_pred_prob, n_samples):
    origin_index = np.arange(0, len(y_pred_prob))
    entropy = -np.nansum(np.multiply(y_pred_prob, np.log(y_pred_prob)), axis=1)
    pred_label = np.argmax(y_pred_prob, axis=1)
    eni = np.column_stack((origin_index,
                           entropy,
                           pred_label))

    eni = eni[(-eni[:, 1]).argsort()]
    return eni[:n_samples], eni[:, 0].astype(int)[:n_samples]

In [0]:
def get_high_confidence_samples(y_pred_prob, delta):
    eni, eni_idx = entropy(y_pred_prob, len(y_pred_prob))
    hcs = eni[eni[:, 1] < delta]
    return hcs[:, 0].astype(int), hcs[:, 2].astype(int)

In [0]:
def get_uncertain_samples(y_pred_prob, n_samples, criteria):
    if criteria == 'lc':
        return least_confidence(y_pred_prob, n_samples)
    elif criteria == 'ms':
        return margin_sampling(y_pred_prob, n_samples)
    elif criteria == 'en':
        return entropy(y_pred_prob, n_samples)
    elif criteria == 'rs':
        return None, random_sampling(y_pred_prob, n_samples)
    else:
        raise ValueError(
            'Unknown criteria value \'%s\', use one of [\'rs\',\'lc\',\'ms\',\'en\']' % criteria)

In [0]:
def run_ceal(X_train, X_test, y_train, y_test,
             model, maximum_iterations, cost_effective, verbose, 
             uncertain_samples_size, uncertain_criteria, 
             delta, threshold_decay, fine_tuning_interval,
             epochs, batch_size, earlystop):
    
    X_pool, y_pool, X_initial, y_initial, X_test, y_test = split_dataset(X_train, X_test, y_train, y_test, 0.1)    
    
    print('X_pool.shape:', X_pool.shape)
    print('y_pool.shape:', y_pool.shape)
    print('X_initial.shape:', X_initial.shape)
    print('y_initial.shape:', y_initial.shape)
    print('X_test.shape:', X_test.shape)
    print('y_test.shape:', y_test.shape)
    
    model = initialize_model(model, X_initial, y_initial, X_test, y_test, 
                             n_classes, epochs, batch_size, verbose)

    w, h, c = X_pool[-1,].shape

    # unlabeled samples
    DU = X_pool, y_pool

    # initially labeled samples
    DL = X_initial, y_initial

    # high confidence samples
    DH = np.empty((0, w, h, c)), np.empty((0, n_classes))

    for i in range(maximum_iterations):

        y_pred_prob = model.predict(DU[0], verbose=verbose)

        _, un_idx = get_uncertain_samples(y_pred_prob, uncertain_samples_size, criteria=uncertain_criteria)
        DL = np.append(DL[0], np.take(DU[0], un_idx, axis=0), axis=0), \
             np.append(DL[1], np.take(DU[1], un_idx, axis=0), axis=0)

        if cost_effective:
            hc_idx, hc_labels = get_high_confidence_samples(y_pred_prob, delta)
            # remove samples also selected through uncertain
            hc = np.array([[i, l] for i, l in zip(hc_idx, hc_labels) if i not in un_idx])
            if hc.size != 0:
                DH = np.take(DU[0], hc[:, 0], axis=0), np_utils.to_categorical(hc[:, 1], n_classes)

        if i % fine_tuning_interval == 0:
            dtrain_x = np.concatenate((DL[0], DH[0])) if DH[0].size != 0 else DL[0]
            dtrain_y = np.concatenate((DL[1], DH[1])) if DH[1].size != 0 else DL[1]

            model.fit(dtrain_x, dtrain_y, validation_data=(X_test, y_test), batch_size=batch_size,
                      shuffle=True, epochs=epochs, verbose=verbose, callbacks=[earlystop])
            delta -= (threshold_decay * fine_tuning_interval)

        DU = np.delete(DU[0], un_idx, axis=0), np.delete(DU[1], un_idx, axis=0)
        DH = np.empty((0, w, h, c)), np.empty((0, n_classes))

        _, acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=verbose)
        
        print(
            'Iteration: %d; High Confidence Samples: %d; Uncertain Samples: %d; Delta: %.5f; Labeled Dataset Size: %d; Accuracy: %.2f'
            % (i, len(DH[0]), len(DL[0]), delta, len(DL[0]), acc))
        
        return model

In [0]:
# keras callbacks
earlystop = EarlyStopping(monitor='val_loss', patience=1)

model = run_ceal(X_train=X_train, 
                 X_test=X_test, 
                 y_train=y_train, 
                 y_test=y_test,
                 model=model,
                 maximum_iterations=10, 
                 cost_effective=True, 
                 verbose=1, 
                 uncertain_samples_size=10,
                 uncertain_criteria='lc', 
                 delta=0.05, 
                 threshold_decay=0.0033, 
                 fine_tuning_interval=1,
                 epochs=5,
                 batch_size=2, 
                 earlystop=earlystop)

TypeError: ignored

In [0]:
class Converge(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        return
 
    def on_train_end(self, logs={}):
        return
 
    def on_epoch_begin(self, epoch, logs={}):
        return
 
    def on_epoch_end(self, epoch, logs={}):
      if logs['acc'] > 0.80:
          self.model.stop_training = True
          print(f"Stopping after {epoch+1} epochs.")

    def on_batch_begin(self, batch, logs={}):
        return
 
    def on_batch_end(self, batch, logs={}):
        return

NameError: ignored

## VGG U-Net

In [0]:
print("Model output shape" , m.output_shape)

Model output shape (None, 12544, 10)


In [0]:
G = imageSegmentationGenerator(X_train_path, y_train_path, batch_size, n_classes, input_height, input_width, m.outputHeight, m.outputWidth)

In [0]:
G_val = imageSegmentationGenerator(X_val_path, y_val_path, batch_size, n_classes, input_height, input_width, m.outputHeight, m.outputWidth)

In [0]:
G

<generator object imageSegmentationGenerator at 0x7f07f05d2a40>

In [0]:
start_time = time.time()

m.fit_generator(generator=G, steps_per_epoch=512, 
                validation_data=G_val, validation_steps=200, 
                epochs=15, use_multiprocessing=True)

m.save_weights(base_dir + 'vggnet_weights')
m.save(base_dir + 'vggnet_weights' + ".vggunet_model")

elapsed_time = time.time() - start_time
print('VGG U-Net training time:', elapsed_time)

## FCN32

In [0]:
def FCN32(n_classes, weights_path, input_height, input_width, image_ordering='channels_first'):
  
    assert input_height % 32 == 0
    assert input_width % 32 == 0

    img_input = Input(shape=(3, input_height, input_width))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=image_ordering)(
        img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=image_ordering)(x)
    f1 = x
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=image_ordering)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=image_ordering)(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=image_ordering)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=image_ordering)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=image_ordering)(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=image_ordering)(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=image_ordering)(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1000, activation='softmax', name='predictions')(x)

    vgg = Model(img_input, x)
    vgg.load_weights(weights_path)

    o = f5

    o = (Conv2D(4096, (7, 7), activation='relu', padding='same', data_format=image_ordering))(o)
    o = Dropout(0.5)(o)
    o = (Conv2D(4096, (1, 1), activation='relu', padding='same', data_format=image_ordering))(o)
    o = Dropout(0.5)(o)

    o = (Conv2D(n_classes, (1, 1), kernel_initializer='he_normal', data_format=image_ordering))(o)
    o = Conv2DTranspose(n_classes, kernel_size=(64, 64), 
                        strides=(32, 32), use_bias=False, 
                        data_format=image_ordering)(o)
    o_shape = Model(img_input, o).output_shape

    outputHeight = o_shape[2]
    outputWidth = o_shape[3]

    print("koko", o_shape)

    o = (Reshape((-1, outputHeight * outputWidth)))(o)
    o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model(img_input, o)
    model.outputWidth = outputWidth
    model.outputHeight = outputHeight

    return model

In [0]:
m = FCN32(10, 
          VGG_weights_path, 
          input_height, 
          input_width)

m.compile(loss = 'categorical_crossentropy',
          optimizer = 'adam',
          metrics = ['accuracy'])

koko (None, 10, 256, 256)


In [0]:
m.summary()

In [0]:
m.fit_generator(generator=G, steps_per_epoch=100, 
                validation_data=G_val, validation_steps=200, 
                epochs=15, use_multiprocessing=True)

m.save_weights(base_dir + 'vggnet_weights')
m.save(base_dir + 'vggnet_weights' + ".fcn32_model")

Epoch 1/15
100/100 [==============================] - 27s 273ms/step - loss: 0.9268 - acc: 0.6208 - val_loss: 1.1054 - val_acc: 0.5850
Epoch 2/15
100/100 [==============================] - 27s 272ms/step - loss: 0.7578 - acc: 0.7165 - val_loss: 1.9422 - val_acc: 0.3922
Epoch 3/15
100/100 [==============================] - 27s 273ms/step - loss: 0.8793 - acc: 0.6593 - val_loss: 1.3506 - val_acc: 0.4661
Epoch 4/15
100/100 [==============================] - 27s 273ms/step - loss: 0.7235 - acc: 0.7293 - val_loss: 1.1284 - val_acc: 0.5539
Epoch 5/15
100/100 [==============================] - 27s 272ms/step - loss: 0.8706 - acc: 0.6615 - val_loss: 1.9203 - val_acc: 0.3664
Epoch 6/15
100/100 [==============================] - 27s 271ms/step - loss: 0.7564 - acc: 0.7021 - val_loss: 1.2095 - val_acc: 0.5487
Epoch 7/15
100/100 [==============================] - 27s 272ms/step - loss: 0.8000 - acc: 0.7031 - val_loss: 1.0328 - val_acc: 0.6319
Epoch 8/15
100/100 [==============================] - 2

## FCN8

In [0]:
# Crop o1 with respect to o2
def crop(o1, o2, i, image_ordering='channels_first'):
    o_shape2 = Model(i, o2).output_shape
    outputHeight2 = o_shape2[2]
    outputWidth2 = o_shape2[3]

    o_shape1 = Model(i, o1).output_shape
    outputHeight1 = o_shape1[2]
    outputWidth1 = o_shape1[3]

    cx = abs(outputWidth1 - outputWidth2)
    cy = abs(outputHeight2 - outputHeight1)

    if outputWidth1 > outputWidth2:
        o1 = Cropping2D(cropping=((0, 0), (0, cx)), data_format=image_ordering)(o1)
    else:
        o2 = Cropping2D(cropping=((0, 0), (0, cx)), data_format=image_ordering)(o2)

    if outputHeight1 > outputHeight2:
        o1 = Cropping2D(cropping=((0, cy), (0, 0)), data_format=image_ordering)(o1)
    else:
        o2 = Cropping2D(cropping=((0, cy), (0, 0)), data_format=image_ordering)(o2)

    return o1, o2

In [0]:
def FCN8(nClasses, weights_path, input_height, input_width, image_ordering='channels_first'):

    img_input = Input(shape=(3, input_height, input_width))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format=image_ordering)(
        img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format=image_ordering)(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format=image_ordering)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format=image_ordering)(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format=image_ordering)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format=image_ordering)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format=image_ordering)(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format=image_ordering)(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format=image_ordering)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format=image_ordering)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format=image_ordering)(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1000, activation='softmax', name='predictions')(x)

    vgg = Model(img_input, x)
    vgg.load_weights(weights_path)

    o = f5
    o = (Conv2D(4096, (7, 7), activation='relu', padding='same', data_format=image_ordering))(o)
    o = Dropout(0.5)(o)
    o = (Conv2D(4096, (1, 1), activation='relu', padding='same', data_format=image_ordering))(o)
    o = Dropout(0.5)(o)
    o = (Conv2D(nClasses, (1, 1), kernel_initializer='he_normal', data_format=image_ordering))(o)
    o = Conv2DTranspose(nClasses, kernel_size=(4, 4), strides=(2, 2), use_bias=False, data_format=image_ordering)(o)
    o2 = f4
    o2 = (Conv2D(nClasses, (1, 1), kernel_initializer='he_normal', data_format=image_ordering))(o2)

    o, o2 = crop(o, o2, img_input, image_ordering)
    o = Add()([o, o2])
    o = Conv2DTranspose(nClasses, kernel_size=(4, 4), strides=(2, 2), use_bias=False, data_format=image_ordering)(o)
    o2 = f3
    o2 = (Conv2D(nClasses, (1, 1), kernel_initializer='he_normal', data_format=image_ordering))(o2)
    
    o2, o = crop(o2, o, img_input, image_ordering)
    o = Add()([o2, o])

    o = Conv2DTranspose(nClasses, kernel_size=(16, 16), strides=(8, 8), use_bias=False, data_format=image_ordering)(o)

    o_shape = Model(img_input, o).output_shape

    outputHeight = o_shape[2]
    outputWidth = o_shape[3]

    o = (Reshape((-1, outputHeight * outputWidth)))(o)
    o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model(img_input, o)
    model.outputWidth = outputWidth
    model.outputHeight = outputHeight

    return model

In [0]:
m = FCN8(10, 
         VGG_weights_path, 
         input_height, 
         input_width)

m.compile(loss = 'categorical_crossentropy',
          optimizer = 'adam',
          metrics = ['accuracy'])

In [0]:
start_time = time.time()

m.fit_generator(generator=G, steps_per_epoch=100, 
                validation_data=G_val, validation_steps=200, 
                epochs=15, use_multiprocessing=True)

m.save_weights(base_dir + 'vggnet_weights')
m.save(base_dir + 'vggnet_weights' + ".fcn8_model")

elapsed_time = time.time() - start_time
print('FCN8 training time:', elapsed_time)

Epoch 1/15
100/100 [==============================] - 95s 947ms/step - loss: 10.4528 - acc: 0.3152 - val_loss: 10.4180 - val_acc: 0.3189
Epoch 2/15
100/100 [==============================] - 86s 857ms/step - loss: 9.7570 - acc: 0.3553 - val_loss: 10.7778 - val_acc: 0.2997
Epoch 3/15
100/100 [==============================] - 86s 856ms/step - loss: 11.1892 - acc: 0.2745 - val_loss: 12.3560 - val_acc: 0.1987
Epoch 4/15
100/100 [==============================] - 86s 857ms/step - loss: 12.1863 - acc: 0.1991 - val_loss: 12.3605 - val_acc: 0.1987
Epoch 5/15
100/100 [==============================] - 86s 857ms/step - loss: 12.1931 - acc: 0.2192 - val_loss: 12.3599 - val_acc: 0.1987
Epoch 6/15
100/100 [==============================] - 86s 857ms/step - loss: 12.1332 - acc: 0.1979 - val_loss: 12.3584 - val_acc: 0.1987
Epoch 7/15
100/100 [==============================] - 86s 857ms/step - loss: 12.2762 - acc: 0.2185 - val_loss: 12.3577 - val_acc: 0.1987
Epoch 8/15
100/100 [======================

## VGG SegNet

In [0]:
def VGGSegNet(n_classes, weights_path, input_height, input_width, vgg_level=3):
    img_input = Input(shape=(3, input_height, input_width))

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', data_format='channels_first')(
        img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2', data_format='channels_first')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool', data_format='channels_first')(x)
    f1 = x
    
    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1', data_format='channels_first')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2', data_format='channels_first')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool', data_format='channels_first')(x)
    f2 = x

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1', data_format='channels_first')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2', data_format='channels_first')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3', data_format='channels_first')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool', data_format='channels_first')(x)
    f3 = x

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1', data_format='channels_first')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2', data_format='channels_first')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3', data_format='channels_first')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool', data_format='channels_first')(x)
    f4 = x

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1', data_format='channels_first')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2', data_format='channels_first')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3', data_format='channels_first')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool', data_format='channels_first')(x)
    f5 = x

    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1000, activation='softmax', name='predictions')(x)

    vgg = Model(img_input, x)
    vgg.load_weights(weights_path)

    levels = [f1, f2, f3, f4, f5]

    o = levels[vgg_level]

    o = (ZeroPadding2D((1, 1), data_format='channels_first'))(o)
    o = (Conv2D(512, (3, 3), padding='valid', data_format='channels_first'))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format='channels_first'))(o)
    o = (ZeroPadding2D((1, 1), data_format='channels_first'))(o)
    o = (Conv2D(256, (3, 3), padding='valid', data_format='channels_first'))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format='channels_first'))(o)
    o = (ZeroPadding2D((1, 1), data_format='channels_first'))(o)
    o = (Conv2D(128, (3, 3), padding='valid', data_format='channels_first'))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format='channels_first'))(o)
    o = (ZeroPadding2D((1, 1), data_format='channels_first'))(o)
    o = (Conv2D(64, (3, 3), padding='valid', data_format='channels_first'))(o)
    o = (BatchNormalization())(o)

    o = Conv2D(n_classes, (3, 3), padding='same', data_format='channels_first')(o)
    o_shape = Model(img_input, o).output_shape
    outputHeight = o_shape[2]
    outputWidth = o_shape[3]

    o = (Reshape((-1, outputHeight * outputWidth)))(o)
    o = (Permute((2, 1)))(o)
    o = (Activation('softmax'))(o)
    model = Model(img_input, o)
    model.outputWidth = outputWidth
    model.outputHeight = outputHeight

    return model

In [0]:
m = VGGSegNet(10, 
              VGG_weights_path, 
              input_height, 
              input_width)

m.compile(loss = 'categorical_crossentropy',
          optimizer = 'adam',
          metrics = ['accuracy'])

In [0]:
m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3, 224, 224)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 224, 224)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 224, 224)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 112, 112)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 112, 112)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 112, 112)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 56, 56)       0         
__________

In [0]:
m.fit_generator(generator=G, steps_per_epoch=100, 
                validation_data=G_val, validation_steps=200, 
                epochs=15, use_multiprocessing=True)

m.save_weights(base_dir + 'vggnet_weights')
m.save(base_dir + 'vggnet_weights' + ".vgg_segnet_model")

Epoch 1/15
100/100 [==============================] - 27s 272ms/step - loss: 0.8889 - acc: 0.6288 - val_loss: 1.0426 - val_acc: 0.6414
Epoch 2/15
100/100 [==============================] - 27s 269ms/step - loss: 0.7134 - acc: 0.7310 - val_loss: 2.5854 - val_acc: 0.3728
Epoch 3/15
100/100 [==============================] - 27s 271ms/step - loss: 0.8679 - acc: 0.6508 - val_loss: 0.8867 - val_acc: 0.6663
Epoch 4/15
100/100 [==============================] - 27s 270ms/step - loss: 0.6987 - acc: 0.7354 - val_loss: 0.9369 - val_acc: 0.6285
Epoch 5/15
100/100 [==============================] - 27s 270ms/step - loss: 0.8566 - acc: 0.6582 - val_loss: 0.9442 - val_acc: 0.6352
Epoch 6/15
100/100 [==============================] - 27s 272ms/step - loss: 0.7363 - acc: 0.7112 - val_loss: 1.1906 - val_acc: 0.5189
Epoch 7/15
100/100 [==============================] - 27s 270ms/step - loss: 0.7734 - acc: 0.7029 - val_loss: 1.2015 - val_acc: 0.6056
Epoch 8/15
100/100 [==============================] - 2

KeyboardInterrupt: ignored

In [0]:
m.fit_generator(generator=G, steps_per_epoch=512, 
                validation_data=G_val, validation_steps=200, 
                epochs=15, use_multiprocessing=True)

m.save_weights(base_dir + 'vggnet_weights')
m.save(base_dir + 'vggnet_weights' + ".vgg_segnet_model")

Epoch 1/15
512/512 [==============================] - 91s 177ms/step - loss: 0.7503 - acc: 0.7020 - val_loss: 1.1700 - val_acc: 0.6248
Epoch 2/15
512/512 [==============================] - 90s 177ms/step - loss: 0.7153 - acc: 0.7153 - val_loss: 0.8926 - val_acc: 0.6558
Epoch 3/15
512/512 [==============================] - 90s 176ms/step - loss: 0.6796 - acc: 0.7423 - val_loss: 1.6742 - val_acc: 0.4541
Epoch 4/15
512/512 [==============================] - 90s 176ms/step - loss: 0.6369 - acc: 0.7663 - val_loss: 0.8517 - val_acc: 0.7180
Epoch 5/15
512/512 [==============================] - 90s 176ms/step - loss: 0.5921 - acc: 0.7923 - val_loss: 0.7160 - val_acc: 0.7604
Epoch 6/15
512/512 [==============================] - 89s 175ms/step - loss: 0.5581 - acc: 0.8029 - val_loss: 0.7444 - val_acc: 0.7396
Epoch 7/15
512/512 [==============================] - 90s 175ms/step - loss: 0.5186 - acc: 0.8141 - val_loss: 0.7548 - val_acc: 0.7440
Epoch 8/15
512/512 [==============================] - 9